#### Code used to seperate questions from combined generated question response of LLM

In [12]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd

file_path = 'Eval_Model_Input.xlsx'

df = pd.read_excel(file_path)

In [14]:
from api import chat_with_model, SUPPORTED_MODELS

In [15]:
print(SUPPORTED_MODELS)

['gemma3:27b', 'mixtral:latest', 'mistral:latest', 'deepseek-r1:70b', 'llama3.3:latest', 'deepseek-r1:14b', 'phi4:latest', 'gemma3:latest', 'mistral-small:22b', 'phi4-mini:latest', 'granite3.2:2b', 'llama3.2:latest']


In [16]:
default_options = {
    "temperature": 0.8,
    "top_p": 0.9,
    "top_k": 40,
    "num_predict": -1,
    "presence_penalty": 0.0,
    "frequency_penalty": 0.0,
    "repeat_penalty": 1.1,
    "stop": [],
    "seed": None,
    "mirostat": 0,
    "mirostat_tau": 5.0,
    "mirostat_eta": 0.1,
    "grammar": None
}

In [17]:
system_prompt = """
You are a helpful data parser who helps me separate the sections for Remember, Understand, Apply, Analyze, Evaluate, and Create from the combined text. Convert this into a JSON with only 6 items mapping to each without rewording or changing anything else and no content should be omitted and there should be no header, leading or styling or trailing data usually generated by llms and should be committed.
Beware of special characters and make sure everything is pertaining to ASCII.
The structure should be like
{
"Remember": content,
"Understand": content,
"Apply": content,
"Analyze": content,
"Evaluate": content,
"Create": content
}
"""
def llm_inference(prompt, model = "gemma3:latest"):
    filled_prompt = f"This is the combined text. Extract all the sections and give me the JSON. Don't reword, and don't omit anything. You just need to split it at the right position\n: {prompt}"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": filled_prompt}
    ]
    
    try:
        response = chat_with_model(
            model_name=model,
            messages=messages,
            options=default_options
        )
        answer = response["message"]["content"].strip()
    except Exception as e:
        answer = f"Error: {e}"
    
    return answer

In [27]:
import csv
import json
if df.shape[1] > 3:
    for value in df.iloc[:, 3]:
        response = llm_inference(value).strip()
        json_string = response[7:-3]
        data = json.loads(json_string)
        
        # combined_data = {key: ' '.join(value_list) for key, value_list in data.items()}
        
        with open('output.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for k, v in data.items():
                print(f"{v.strip()}", end="\n\n\n")
                writer.writerow([v.strip()])
        # def test_code():
        #     try:
        #         response = llm_inference(value).strip()
        #         json_string = response
        #         data = json.loads(json_string)
                
        #         combined_data = {key: ' '.join(value_list) for key, value_list in data.items()}
                
                
        #         with open('output.csv', 'a', newline='') as csvfile:
        #             writer = csv.writer(csvfile)
        #             for k, v in combined_data.items():
        #                 print(f"{v}\n")
        #                 writer.writerow([v.strip()])
        #     except:
        #         print(f"**********************Exception:**********************\n{value}", end="\n\n\n")
        #         test_code()
        # test_code()
else:
    print("Column 'D' does not exist in the file.")

What is a decision tree model? Can you recall its basic structure and components (root node, internal nodes, leaf nodes)? How do they represent decision-making processes in a simple, visual manner?


Describe the role of nodes with conditional statements in a decision tree. When would an incoming data point typically move from one branch to another based on these conditions? Can you explain how splitting criteria like Gini index or information gain help in forming these branches?


Suppose an Indian textile manufacturing company wants to decide whether to invest in a new, eco-friendly spinning technology for their factories due to its potential market demand and high setup costs. Apply the decision tree model principles to sketch out the initial tree structure. Identify what variables (like fiber type, production volume, target market, etc.) could be at the root node, and then suggest potential conditions or criteria that might lead to further branches for 'yes' (invest) or 'no' (do no

JSONDecodeError: Invalid control character at: line 3 column 769 (char 771)

In [18]:
pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
# Block: Setup OpenAI API and GitHub Repo Access
import openai
from openai import OpenAI

client = None
# Function to set OpenAI API Key
def set_openai_api_key(api_key):
  global client
  client = OpenAI(api_key=api_key)
  openai.api_key = api_key

In [ ]:
key = "<MY_OPENAI_API_KEY>"
# Set the OpenAI API key
set_openai_api_key(key)

In [21]:
# Block: Send Prompt to OpenAI ChatGPT API
def global_openai_api_call(prompt, engine='gpt-4o'):
    messages = [
    {"role": "system", "content": system_prompt}
    ]
    # Call the OpenAI API to generate content
    global client
    temp_Client = client
    messages.append({"role": "user", "content": prompt})

    try:
      response = temp_Client.completions.create(
        model=engine,
        prompt=prompt,
        temperature=0.1
      )
    except openai.APIError as e:
      try:
        response = client.chat.completions.create(
          model=engine,
          messages=messages
          )
        return response.choices[0].message.content.strip()
      except Exception as e:
        print(f"Error: {e}")
        return None, None
    except Exception as e:
      print(f"Error: {e}")
    return response.choices[0].text.strip()

In [22]:
import csv
import json
if df.shape[1] > 3:
    for value in df.iloc[:, 3]:
        response_str = global_openai_api_call(value)
        json_string = response_str.lstrip("```json").lstrip("```").rstrip("```").strip()
        print(json_string)
        data = json.loads(json_string)
        
        # combined_data = {key: ' '.join(value_list) for key, value_list in data.items()}
        
        with open('output.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for k, v in data.items():
                if v is not str:
                    v = str(v)
                print(f"{v.strip()}", end="\n\n\n")
                writer.writerow([v.strip()])
else:
    print("Column 'D' does not exist in the file.")

{
  "Remember": "What is a decision tree model? Can you recall its basic structure and components (root node, internal nodes, leaf nodes)? How do they represent decision-making processes in a simple, visual manner?",
  "Understand": "Describe the role of nodes with conditional statements in a decision tree. When would an incoming data point typically move from one branch to another based on these conditions? Can you explain how splitting criteria like Gini index or information gain help in forming these branches?",
  "Apply": "Suppose an Indian textile manufacturing company wants to decide whether to invest in a new, eco-friendly spinning technology for their factories due to its potential market demand and high setup costs. Apply the decision tree model principles to sketch out the initial tree structure. Identify what variables (like fiber type, production volume, target market, etc.) could be at the root node, and then suggest potential conditions or criteria that might lead to furt